In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/panduan1.json').read()
intents = json.loads(data_file)

# Tokenize dan proses data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenisasi setiap kata
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Menambahkan dokumen
        documents.append((w, intent['tag']))
        # Menambahkan tag ke dalam classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize dan sort kata dan classes
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Simpan words dan classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Inisialisasi data pelatihan
training = []
output_empty = [0] * len(classes)

# Membuat training data
for doc in documents:
    bag = []
    pattern_words = doc[0]
    # Lemmatize setiap kata
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Buat bag of words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Buat output_row untuk kelas
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Acak dan ubah menjadi numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

# Buat list train_x dan train_y
train_x = np.array([elem[0] for elem in training])
train_y = np.array([elem[1] for elem in training])

print("Training data created")

# Membangun model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Kompilasi model dengan learning_rate yang benar
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit model dan simpan
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


843 documents
51 classes ['TimahTechBot', 'cara-membagikan-file', 'cara-membuat-file-baru', 'cara-membuka-mengedit-file', 'cara-mencetak-file', 'cara_approve_transaksi', 'edit_data_diri', 'goodbye', 'greeting', 'isi_baterai_receiver', 'koneksi_perangkat_smart_tv', 'login_eumd', 'melihat_mengedit_hapus_PR', 'mengubah_password', 'menu_umum_eumd', 'mfa_activation_guide', 'modul_eumd', 'oracle_cloud_learning', 'oracle_cloud_learning_functions', 'oracle_netsuite_login', 'panduan-zoom1', 'panduan-zoom2', 'panduan-zoom3', 'panduan-zoom4', 'panduan-zoom5', 'panduan-zoom6', 'panduan-zoom7', 'panduan_scan_printer', 'panduanfotocopy', 'panduanprinter&scaner1', 'panduanprinter&scanner2', 'panduanprinter&scanner4', 'panduanteams1', 'panduanteams2', 'panduanteams3', 'panduanteams4', 'panduanteams5', 'panduanteams6', 'panduanteams7', 'panduanteams8', 'panduanteams9', 'pemasangan_infocus', 'pemasangan_perangkat_meeting', 'pembuatan_npp_pembayaran', 'pengajuan_uang_muka', 'perawatan_perangkat_meeting',

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0325 - loss: 3.8950
Epoch 2/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1369 - loss: 3.4765
Epoch 3/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2629 - loss: 2.8890
Epoch 4/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3230 - loss: 2.4522
Epoch 5/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4332 - loss: 2.0164
Epoch 6/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4461 - loss: 1.9168
Epoch 7/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4887 - loss: 1.5768
Epoch 8/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5833 - loss: 1.4063
Epoch 9/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5930 - loss: 1.2337
Epoch 10/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6138 - loss: 1.2403
Epoch 11/200
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6582 - loss: 1.0651
Epoch 12/200
169/169 ━━━━━━━━━━━━━━━━━━━━

model created


In [5]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.5 MB/s eta 0:00:00


In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

# Load model and other files
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

# Load intents, words, and classes
intents = json.loads(open('panduan1.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Preprocess the input sentence
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Bag of words model
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Predict the intent of the input sentence
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

# Get a response based on the predicted intent
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

# Generate a chatbot response
def chatbot_response(msg):
    ints = predict_class(msg, model)
    return getResponse(ints, intents)

# Import and set up Gradio
import gradio as gr

# Define the Gradio interface
def chatbot_interface(user_input):
    return chatbot_response(user_input)

# Launch the Gradio interface
interface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="Chatbot")
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5862d59766945e5505.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

#from keras.models import load_model
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('panduan1.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res



#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [5]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import random
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
import string

# Unduh resource yang diperlukan
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Membaca file JSON
with open('/content/panduan1.json') as content:
    data1 = json.load(content)

# Variabel untuk menyimpan data
tags = []  # data tag
inputs = []  # data input atau pattern
responses = {}  # data respon
words = []  # Data kata
classes = []  # Data Kelas atau Tag
documents = []  # Data Kalimat Dokumen
ignore_words = ['?', '!']  # Mengabaikan tanda spesial karakter

# Fungsi untuk mendapatkan sinonim
def get_synonyms(word):
    """Mengambil sinonim dari kata menggunakan WordNet"""
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.append(synonym)
    return synonyms

# Fungsi untuk mengganti kata dalam kalimat dengan sinonim
def synonym_replacement(sentence, n=2):
    """Mengganti n kata dengan sinonim di dalam kalimat"""
    words = sentence.split()
    new_sentence = words.copy()

    words_with_synonyms = [word for word in words if len(get_synonyms(word)) > 0]

    if len(words_with_synonyms) == 0:
        return sentence  # Jika tidak ada kata yang memiliki sinonim, kembalikan kalimat aslinya

    # Pilih kata secara acak untuk diganti sinonimnya
    words_to_replace = random.sample(words_with_synonyms, min(n, len(words_with_synonyms)))

    for word in words_to_replace:
        synonyms = get_synonyms(word)
        if synonyms:
            new_sentence[words.index(word)] = random.choice(synonyms)

    return ' '.join(new_sentence)

# Cek jika kunci 'intents' ada di data JSON
if 'intents' in data1:
    for intent in data1['intents']:  # Iterasi melalui setiap intent di 'intents'
        tag = intent['tag']  # Mendapatkan tag dari intent
        patterns = intent['patterns']  # Mendapatkan patterns dari intent

        responses[tag] = intent['responses']  # Menyimpan responses untuk setiap tag

        for pattern in patterns:  # Iterasi melalui setiap kalimat pattern
            inputs.append(pattern)
            tags.append(tag)

            # Augmentasi dengan sinonim
            augmented_pattern = synonym_replacement(pattern, n=2)
            inputs.append(augmented_pattern)  # Menambahkan pattern yang sudah di-augmentasi

            # Tokenisasi teks (mengubah teks menjadi kata-kata)
            w = nltk.word_tokenize(pattern)
            words.extend(w)
            documents.append((w, tag))

            if tag not in classes:
                classes.append(tag)

# Pastikan inputs dan tags memiliki panjang yang sama
print(f'Panjang inputs: {len(inputs)}')
print(f'Panjang tags: {len(tags)}')

# Membuat DataFrame
if len(inputs) == len(tags):
    data = pd.DataFrame({"patterns": inputs, "tags": tags})
else:
    print("Input dan tag memiliki panjang yang berbeda.")

# Preprocessing
# Menghapus tanda baca pada patterns
data['patterns'] = data['patterns'].apply(lambda wrd: [ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))

# Lemmatization
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print(len(words), "unique lemmatized words", words)

# Sorting data classes
classes = sorted(list(set(classes)))
print(len(classes), "classes", classes)

# Documents = kombinasi antara pattern dan tag dalam intents json
print(len(documents), "documents")

# Tokenizing data (Tokenisasi Data)
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])

# Padding sequences
x_train = pad_sequences(train)
print("Padded sequences:", x_train)

# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])
print("Encoded labels:", y_train)

# Melihat input shape
input_shape = x_train.shape[1]
print("Input shape:", input_shape)

# Vocabulary size
vocabulary = len(tokenizer.word_index)
print("Number of unique words: ", vocabulary)

# Output length (jumlah kelas/tag)
output_length = le.classes_.shape[0]
print("Output length: ", output_length)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Panjang inputs: 1686
Panjang tags: 843
Input dan tag memiliki panjang yang berbeda.


NameError: name 'data' is not defined